#WITHOUT USING PIPELINE

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [ ]:
df=pd.read_csv("/content/loan_approval_dataset.csv")
df

,Age,Salary,Credit_Score,Loan_Amount,Loan_Term,Employment_Status,Residence_Type,Previous_Default,Loan_Approved
0,56,136748,584,38209,36 months,Employed,Owned,Yes,Yes
1,46,25287,815,27424,24 months,Self-Employed,Rented,No,Yes
2,32,146593,398,42396,12 months,Unemployed,Rented,Yes,Yes
3,60,54387,696,11370,24 months,Unemployed,Owned,No,No
4,25,28512,788,14528,12 months,Employed,Owned,No,No
...,...,...,...,...,...,...,...,...,...
995,22,49241,500,41020,24 months,Self-Employed,Owned,No,Yes
996,40,116214,423,12415,48 months,Self-Employed,Owned,No,Yes
997,27,64569,300,28155,36 months,Self-Employed,Rented,Yes,Yes
998,61,31745,490,48884,12 months,Self-Employed,Mortgage,No,Yes


In [ ]:
x=df.drop(columns=['Loan_Approved'])
y=df['Loan_Approved']

In [ ]:
num_col=x.select_dtypes(include='number').columns
cat_col=x.select_dtypes(exclude='number').columns

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)

In [ ]:
def num_encode_concat(df):
  num_col=xtrain.select_dtypes(include='number')
  obj_col=xtrain.select_dtypes(exclude='number')
  onehot=OneHotEncoder(sparse_output=False)
  obj_encoded_cols=onehot.fit_transform(obj_col)
  new_obj_cols=pd.DataFrame(obj_encoded_cols,columns=onehot.get_feature_names_out())
  new_df=pd.concat([num_col.reset_index(drop=True),new_obj_cols.reset_index(drop=True)],axis=1)
  return new_df

In [ ]:
xtrain = num_encode_concat(xtrain)
xtest = num_encode_concat(xtest)

In [ ]:
# compose=ColumnTransformer(transformers=[('scaling',StandardScaler(),num_col),('encoding',OneHotEncoder(),cat_col)])
# compose.fit_transform(df)

In [ ]:
grid_search_cv=GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid={'C':[0.01,0.1,1.0,10],
                'penalty':['l1','l2'],
                'solver':['liblinear'],
                'class_weight':['balanced',None]
    },
    cv=10,
    n_jobs=-1,
    verbose=1,
    scoring='f1_macro'
)

In [ ]:
grid_search_cv.fit(xtrain,ytrain)

Fitting 10 folds for each of 16 candidates, totalling 160 fits


GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1.0, 10],
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
             scoring='f1_macro', verbose=1)

In [ ]:
grid_search_cv.score(xtrain,ytrain)

0.5310815216028486

In [ ]:
grid_search_cv.best_params_

{'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}

In [ ]:
model=grid_search_cv.best_estimator_

In [ ]:
model.fit(xtrain,ytrain)

LogisticRegression(C=0.01, class_weight='balanced', max_iter=1000,
                   solver='liblinear')

In [ ]:
model.score(xtrain,ytrain)

0.5325

In [ ]:
grid_search_cv.best_score_

np.float64(0.49935196977874236)

In [ ]:
grid_search_cv.predict(xtest)

array(['Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes',
       'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No',
       'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No',
       'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
       'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'Yes',
       'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes',
       'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes',
       'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No',
       'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes',
       'Yes', 'No', 'Yes', 'No', 'No', 'No', 'No', 'No', 'No', 'Yes',
       'No', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No',
       'Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'No',
       'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No',
       'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No'

In [ ]:
cv_result=pd.DataFrame(grid_search_cv.cv_results_)

In [ ]:
cv_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_penalty,param_solver,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011278,0.002782,0.009591,0.003484,0.01,balanced,l1,liblinear,"{'C': 0.01, 'class_weight': 'balanced', 'penal...",0.423559,...,0.399625,0.595960,0.499687,0.545455,0.531571,0.483546,0.455610,0.495842,0.057812,3
1,0.009316,0.001741,0.006626,0.000575,0.01,balanced,l2,liblinear,"{'C': 0.01, 'class_weight': 'balanced', 'penal...",0.487420,...,0.398496,0.622879,0.473684,0.466667,0.572596,0.498747,0.461743,0.499352,0.061560,1
2,0.007770,0.001001,0.006425,0.000766,0.01,None,l1,liblinear,"{'C': 0.01, 'class_weight': None, 'penalty': '...",0.446889,...,0.398496,0.587436,0.449656,0.536849,0.548872,0.477624,0.458353,0.486434,0.053194,12
3,0.008904,0.001180,0.006729,0.001446,0.01,None,l2,liblinear,"{'C': 0.01, 'class_weight': None, 'penalty': '...",0.461743,...,0.424640,0.587436,0.487420,0.494949,0.475000,0.455610,0.474672,0.487299,0.043121,11
4,0.008772,0.000886,0.006579,0.000809,0.10,balanced,l1,liblinear,"{'C': 0.1, 'class_weight': 'balanced', 'penalt...",0.474672,...,0.386637,0.610980,0.473684,0.517460,0.559125,0.473684,0.462916,0.494625,0.057339,4
5,0.008481,0.002464,0.006009,0.000227,0.10,balanced,l2,liblinear,"{'C': 0.1, 'class_weight': 'balanced', 'penalt...",0.475000,...,0.398496,0.622879,0.485490,0.466667,0.572596,0.498747,0.449656,0.498082,0.062385,2
6,0.008115,0.002232,0.006652,0.001476,0.10,None,l1,liblinear,"{'C': 0.1, 'class_weight': None, 'penalty': 'l...",0.461743,...,0.411673,0.587436,0.437412,0.485490,0.536849,0.469697,0.455610,0.479314,0.047623,15
7,0.008761,0.001039,0.006260,0.000656,0.10,None,l2,liblinear,"{'C': 0.1, 'class_weight': None, 'penalty': 'l...",0.449656,...,0.436708,0.562432,0.450000,0.400000,0.474672,0.444444,0.462416,0.466714,0.042147,16
8,0.015452,0.004592,0.006557,0.000937,1.00,balanced,l1,liblinear,"{'C': 1.0, 'class_weight': 'balanced', 'penalt...",0.497172,...,0.486778,0.561884,0.525000,0.487420,0.461743,0.437412,0.446889,0.487853,0.036158,9
9,0.008349,0.001391,0.005966,0.000575,1.00,balanced,l2,liblinear,"{'C': 1.0, 'class_weight': 'balanced', 'penalt...",0.475000,...,0.398496,0.595960,0.473684,0.466667,0.572596,0.498747,0.437412,0.491709,0.059468,8
